**Question/need:** Can we predict the success (gross revenues, revenues per budget dollar, and/or oscar nominations) of proposed low-budget films (< $10 million) based on its characteristics? Are there different predictive characteristics for success in terms of gross revenues vs. oscar nominations? 

**Movie data:** I'm planning on using all movies (1980 - 2016) from boxofficemojo to investigate the impact of various characteristics on success. If I have time, I would also like to incorporate other variables, such as google searches and wikipedia page views, as well as critic and general public rating information from rottentomatoes. 

**Characteristics of each movie and/or other entities:** I'd like to investigate as many characteristics as possible during my first pass to determine which ones have the greatest predictive impact, and then dig in deeper to these characteristics. The features I'm most interested in are: genre, release date (month, if before a holiday weekend, if during Christmas holiday, if during summer), star power (a score that accounts for actors, director, and producers), production budget, and if there the movie has some preexisting popularity, either through a novel/play adaptation or if there's a prequel. I'll also investigate other features such as runtime, rating, and franchise, but I have a hunch that these will have a smaller impact. 

In [13]:
# storing
import pickle

# analysis 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


First, let's load the movie data into a dataframe and check the quality of our data.

In [40]:
with open('pickled_data/all-movies-data.pkl', 'r') as picklefile:
    all_movies_data = pickle.load(picklefile)

In [41]:
with open('pickled_data/failed-urls.pkl', 'r') as picklefile:
    failed_urls = pickle.load(picklefile)

In [42]:
print failed_urls

['http://www.boxofficemojo.com/movies/?id=longshot2013.htm']


In [43]:
all_movies = pd.DataFrame(all_movies_data)

In [47]:
all_movies.head()

,1-title,2-release_date,3-closing_date,actors,budget,director,distributor,dom_total_gross,genre,intl_total_gross,oscar_noms,oscar_wins,producers,rating,runtime_(mins),theaters,url,writers
0,The A-Team,2010-06-11,2010-09-16,"[Liam Neeson, Bradley Cooper, Sharlto Copley, ...",110000000,[Joe Carnahan],Fox,77222099,Action,177238796,0,0,"[Ridley Scott, Tony Scott, Alex Young]",PG-13,117,3544,ateam.htm,[Skip Woods]
1,A.C.O.D.,2013-10-04,2013-11-07,"[Adam Scott, Catherine O'Hara, Richard Jenkins...",NaN,None,The Film Arcade,175705,Comedy,NaN,0,0,[Teddy Schwarzman],PG-13,88,42,acod.htm,None
2,A.I. Artificial Intelligence,2001-06-29,NaT,"[Haley Joel Osment, Frances O'Connor, Jude Law...",100000000,[Steven Spielberg],Warner Bros.,78616689,Sci-Fi,235926552,2,0,"[Kathleen Kennedy, Steven Spielberg]",PG-13,145,3242,ai.htm,None
3,Aaja Nachle,2007-11-30,2007-12-20,None,NaN,None,Yash Raj,484108,Foreign,6773493,0,0,None,Unrated,145,66,aajanachle.htm,None
4,Aarakshan,2011-08-12,2011-09-22,None,NaN,None,Reliance Big Pictures,651096,Foreign,651096,0,0,None,Unrated,NaN,91,aarakshan.htm,None


In [52]:
all_movies.describe()

,budget,dom_total_gross,intl_total_gross,oscar_noms,oscar_wins,runtime_(mins),theaters
count,2.874000e+03,1.406300e+04,6.476000e+03,16100.000000,16100.000000,15163.000000,13009.000000
mean,4.396819e+07,1.800154e+07,7.071239e+07,0.196646,0.043106,104.045242,783.702052
std,4.360149e+07,4.289784e+07,1.500811e+08,0.898730,0.362035,22.042827,1102.616644
min,1.100000e+06,3.000000e+01,1.000000e+00,0.000000,0.000000,35.000000,1.000000
25%,1.400000e+07,6.902100e+04,9.030020e+05,0.000000,0.000000,91.000000,7.000000
50%,3.000000e+07,1.110522e+06,1.259295e+07,0.000000,0.000000,100.000000,74.000000
75%,6.000000e+07,1.629691e+07,7.383256e+07,0.000000,0.000000,112.000000,1395.000000
max,3.000000e+08,8.631482e+08,2.787965e+09,10.000000,9.000000,729.000000,4468.000000


Since we're interested in budget data, let's check to see how many movies of the 16,100 total movies have budget data.

In [51]:
all_movies.budget.count()

2874

Only 2,874 out of 16,100 movies have budget data. That's only 17.9% of the entire dataset! Let's take a closer look at movies with budget data.

In [59]:
# drop movies without budget data 
only_budget = all_movies[pd.notnull(all_movies['budget'])]

# add some roi calculations
only_budget['dom_roi'] = only_budget['dom_total_gross'] / only_budget['budget']
only_budget['intl_roi'] = only_budget['intl_total_gross'] / only_budget['budget']

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Let's look at only low budget films, which we're defining as films under $10m and evaluate the quality of the data.

In [58]:
# get only low budget (<$10m) films
low_budget = only_budget[only_budget['budget'] < 10000000]
low_budget.count()

1-title             469
2-release_date      463
3-closing_date      285
actors              335
budget              469
director            252
distributor         469
dom_total_gross     442
genre               469
intl_total_gross    263
oscar_noms          469
oscar_wins          469
producers           192
rating              469
runtime_(mins)      464
theaters            401
url                 469
writers             175
dtype: int64

Looks like there are 469 total movies in our low budget films dataset.